# سیمینٹک کرنل

اس کوڈ نمونے میں، آپ [سیمینٹک کرنل](https://aka.ms/ai-agents-beginners/semantic-kernel) اے آئی فریم ورک کا استعمال کرتے ہوئے ایک بنیادی ایجنٹ بنائیں گے۔

اس نمونے کا مقصد آپ کو وہ مراحل دکھانا ہے جو ہم بعد میں مختلف ایجنٹک پیٹرنز کو نافذ کرتے وقت اضافی کوڈ نمونوں میں استعمال کریں گے۔


## مطلوبہ پائتھن پیکجز درآمد کریں


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## کلائنٹ بنانا

اس مثال میں، ہم [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) کا استعمال کریں گے تاکہ LLM تک رسائی حاصل کی جا سکے۔

`ai_model_id` کو `gpt-4o-mini` کے طور پر مقرر کیا گیا ہے۔ مختلف نتائج دیکھنے کے لیے ماڈل کو GitHub Models مارکیٹ پلیس میں دستیاب کسی دوسرے ماڈل میں تبدیل کرنے کی کوشش کریں۔

`Azure Inference SDK` استعمال کرنے کے لیے، جو GitHub Models کے `base_url` کے لیے استعمال ہوتا ہے، ہم Semantic Kernel کے اندر `OpenAIChatCompletion` کنیکٹر استعمال کریں گے۔ Semantic Kernel کو دوسرے ماڈل فراہم کنندگان کے لیے استعمال کرنے کے لیے دیگر [دستیاب کنیکٹرز](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) بھی موجود ہیں۔


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## ایجنٹ بنانا

نیچے ہم ایک ایجنٹ بناتے ہیں جس کا نام `TravelAgent` ہے۔

اس مثال میں، ہم بہت سادہ ہدایات استعمال کر رہے ہیں۔ آپ ان ہدایات کو تبدیل کر سکتے ہیں تاکہ دیکھ سکیں کہ ایجنٹ مختلف طریقے سے کیسے ردعمل دیتا ہے۔


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## ایجنٹ کو چلانا

اب ہم ایجنٹ کو `ChatHistoryAgentThread` قسم کے تھریڈ کو ڈیفائن کرکے چلا سکتے ہیں۔ کسی بھی ضروری سسٹم پیغامات ایجنٹ کے invoke_stream کے `messages` کلیدی دلیل میں فراہم کیے جاتے ہیں۔

جب یہ ڈیفائن ہو جائیں، تو ہم ایک `user_inputs` بناتے ہیں جو وہ ہوگا جو صارف ایجنٹ کو بھیج رہا ہے۔ اس مثال میں، ہم نے اس پیغام کو `Plan me a sunny vacation` پر سیٹ کیا ہے۔

آپ اس پیغام کو تبدیل کرنے کے لیے آزاد ہیں تاکہ دیکھ سکیں کہ ایجنٹ مختلف طریقے سے کیسے جواب دیتا ہے۔


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**ڈس کلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے پوری کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستگی ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
